<a href="https://colab.research.google.com/github/clopensesame/Nazzaro_DSPN_S24/blob/main/Exercise_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [1]:
install.packages(c("boot","mlbench"))
library(tidyverse)
library(boot)
library(mlbench)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
data(PimaIndiansDiabetes2)
d <- PimaIndiansDiabetes2
dat <- d %>%
  select(!insulin) %>%
  na.omit()

dim(dat)
head(dat)

[1] 532   8

,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [63]:
dat <- dat %>%
  mutate(diabetes_dic = case_when(diabetes=="neg"~0, diabetes=="pos"~1))

dat_no_first <- dat[-1,]
dat_no_first

#this is log-odds; adding family="binomial" into glm we get [0,0.5)=neg and [0.5,1]=pos i think
glm.fit = glm(diabetes_dic~pregnant+glucose+pressure+triceps+mass+pedigree+age, data=dat_no_first)
summary(glm.fit)

predicted_probs <- predict(glm.fit, newdata = dat[1,], type = "response")
predicted_probs
dat[1,]$diabetes
#prediction=0.65 > 0 so prediction = pos
#actual = pos
#model did correctly classify


#more predictions for obs 2-5
# predicted_probs2 <- predict(glm.fit, newdata = dat[2:5,], type = "response")
# predicted_probs2

,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes,diabetes_dic
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
2,1,85,66,29,26.6,0.351,31,neg,0
4,1,89,66,23,28.1,0.167,21,neg,0
5,0,137,40,35,43.1,2.288,33,pos,1
7,3,78,50,32,31.0,0.248,26,pos,1
9,2,197,70,45,30.5,0.158,53,pos,1
14,1,189,60,23,30.1,0.398,59,pos,1
15,5,166,72,19,25.8,0.587,51,pos,1
17,0,118,84,47,45.8,0.551,31,pos,1
19,1,103,30,38,43.3,0.183,33,neg,0



Call:
glm(formula = diabetes_dic ~ pregnant + glucose + pressure + 
    triceps + mass + pedigree + age, data = dat_no_first)

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.0356375  0.1189484  -8.707  < 2e-16 ***
pregnant     0.0202261  0.0065740   3.077 0.002203 ** 
glucose      0.0060164  0.0005833  10.314  < 2e-16 ***
pressure    -0.0008687  0.0015203  -0.571 0.567991    
triceps      0.0005701  0.0021030   0.271 0.786444    
mass         0.0119056  0.0033206   3.585 0.000368 ***
pedigree     0.1800867  0.0494968   3.638 0.000302 ***
age          0.0041663  0.0021723   1.918 0.055661 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.1470284)

    Null deviance: 117.665  on 530  degrees of freedom
Residual deviance:  76.896  on 523  degrees of freedom
AIC: 498.86

Number of Fisher Scoring iterations: 2


1 
0.6548173

[1] pos
Levels: neg pos

So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [64]:
# Initialize `results` data frame
results = data.frame(actual = dat$diabetes,
                     predicted = NA)

#sanity check
#nrow(results)
#nrow(dat)

#for loop
for (i in 1:nrow(dat)){ #don't forget to change this to your data set name
    # separate individual observation `i` from the rest of your data
    dat_no_i <- dat[-i,]

    # train your model
    glm.fit = glm(diabetes_dic~pregnant+glucose+pressure+triceps+mass+pedigree+age, data=dat_no_i)

    # test model on hold out observation
    predicted_probs <- predict(glm.fit, newdata = dat[i,], type = "response")

    # classify model prediction as "pos" or "neg" and add to `results`
    results$predicted[i] <- ifelse(predicted_probs>0, "pos", "neg")
}
head(results)

,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [90]:
incorrect_count <- sum(results$actual != results$predicted)
incorrect_proportion <- incorrect_count/nrow(results)

incorrect_proportion
#proportion of incorrectly classified cases: 0.58

[1] 0.575188

# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [108]:
glm.fit = glm(diabetes_dic~pregnant+glucose+pressure+triceps+mass+pedigree+age, data=dat)
#don't need to specify r, pi i think
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)

cv.err  = cv.glm(dat, glm.fit, cost, K=nrow(dat))

Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [109]:
#delta[1] == raw mse, delta[2] == adjusted mse
cv.err$delta[1]
#proportion of incorrectly classified cases: 0.22

[1] 0.2199248

How do your results compare to your manual LOOCV above?

> * Manual LOOCV: proportion of incorrectly classified cases was 0.58
> * LOOCV: proportion of incorrectly classified cases was 0.22
> * LOOCV performed much better than Manual LOOCV


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [118]:
set.seed(1)
glm.fit = glm(diabetes_dic~pregnant+glucose+pressure+triceps+mass+pedigree+age, data=dat)
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)

# K = 3
cv.err3  = cv.glm(dat, glm.fit, cost, K=3)

# K = 5
cv.err5  = cv.glm(dat, glm.fit, cost, K=5)

# K = 10
cv.err10  = cv.glm(dat, glm.fit, cost, K=10)

# K = 15
cv.err15  = cv.glm(dat, glm.fit, cost, K=15)

#oh they're similar, is that fine?
cv.err3$delta[1] #error=0.22
cv.err5$delta[1] #error=0.22
cv.err10$delta[1] #error=0.22
cv.err15$delta[1] #error=0.23

[1] 0.2161654

[1] 0.2199248

[1] 0.2236842

[1] 0.2274436

#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * The K=3,5,10,15 errors were pretty similar, and seemed to increase as k increased.

If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> * I would expect to get the same results for LOOCV regardless of the seed, since LOOCV runs on all combinations of the dataset minus one observation. We get different results for k-fold cv since the training set is a random selection of (k-1)/k x nrow(dat) observations.


**DUE:** 5pm March 25, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
